Radek posted about this [here](https://www.kaggle.com/competitions/h-and-m-personalized-fashion-recommendations/discussion/309220), and linked to a GitHub repo with the code.

I just transferred that code here to Kaggle notebooks, that's all.

In [3]:
import numpy as np

def apk(actual, predicted, k=10):
    """
    Computes the average precision at k.

    This function computes the average prescision at k between two lists of
    items.

    Parameters
    ----------
    actual : list
             A list of elements that are to be predicted (order doesn't matter)
    predicted : list
                A list of predicted elements (order does matter)
    k : int, optional
        The maximum number of predicted elements

    Returns
    -------
    score : double
            The average precision at k over the input lists

    """
    if len(predicted)>k:
        predicted = predicted[:k]

    score = 0.0
    num_hits = 0.0

    for i,p in enumerate(predicted):
        if p in actual and p not in predicted[:i]:
            num_hits += 1.0
            score += num_hits / (i+1.0)

    if not actual:
        return 0.0

    return score / min(len(actual), k)

def mapk(actual, predicted, k=10):
    """
    Computes the mean average precision at k.

    This function computes the mean average prescision at k between two lists
    of lists of items.

    Parameters
    ----------
    actual : list
             A list of lists of elements that are to be predicted 
             (order doesn't matter in the lists)
    predicted : list
                A list of lists of predicted elements
                (order matters in the lists)
    k : int, optional
        The maximum number of predicted elements

    Returns
    -------
    score : double
            The mean average precision at k over the input lists

    """
    return np.mean([apk(a,p,k) for a,p in zip(actual, predicted)])

In [4]:
from sklearn.base import BaseEstimator, TransformerMixin
import numpy as np

# https://www.kaggle.com/c/h-and-m-personalized-fashion-recommendations/discussion/308635
def customer_hex_id_to_int(series):
    return series.str[-16:].apply(hex_id_to_int)

def hex_id_to_int(str):
    return int(str[-16:], 16)

def article_id_str_to_int(series):
    return series.astype('int32')

def article_id_int_to_str(series):
    return '0' + series.astype('str')

class Categorize(BaseEstimator, TransformerMixin):
    def __init__(self, min_examples=0):
        self.min_examples = min_examples
        self.categories = []
        
    def fit(self, X):
        for i in range(X.shape[1]):
            vc = X.iloc[:, i].value_counts()
            self.categories.append(vc[vc > self.min_examples].index.tolist())
        return self

    def transform(self, X):
        data = {X.columns[i]: pd.Categorical(X.iloc[:, i], categories=self.categories[i]).codes for i in range(X.shape[1])}
        return pd.DataFrame(data=data)


def calculate_apk(list_of_preds, list_of_gts):
    # for fast validation this can be changed to operate on dicts of {'cust_id_int': [art_id_int, ...]}
    # using 'data/val_week_purchases_by_cust.pkl'
    apks = []
    for preds, gt in zip(list_of_preds, list_of_gts):
        apks.append(apk(gt, preds, k=12))
    return np.mean(apks)

def eval_sub(sub_csv, skip_cust_with_no_purchases=True):
    sub=pd.read_csv(sub_csv)
    validation_set=pd.read_parquet('data/validation_ground_truth.parquet')

    apks = []

    no_purchases_pattern = []
    for pred, gt in zip(sub.prediction.str.split(), validation_set.prediction.str.split()):
        if skip_cust_with_no_purchases and (gt == no_purchases_pattern): continue
        apks.append(apk(gt, pred, k=12))
    return np.mean(apks)

In [5]:
import pandas as pd

In [6]:
%%time

transactions = pd.read_parquet('../input/warmup/transactions_train.parquet')
customers = pd.read_parquet('../input/warmup/customers.parquet')
articles = pd.read_parquet('../input/warmup/articles.parquet')

# sample = 0.05
# transactions = pd.read_parquet(f'data/transactions_train_sample_{sample}.parquet')
# customers = pd.read_parquet(f'data/customers_sample_{sample}.parquet')
# articles = pd.read_parquet(f'data/articles_train_sample_{sample}.parquet')

CPU times: user 2.42 s, sys: 3.82 s, total: 6.24 s
Wall time: 8.84 s


In [7]:
test_week = transactions.week.max() + 1
transactions = transactions[transactions.week > transactions.week.max() - 10]

# Generating candidates

### Last purchase candidates

In [8]:
%%time

c2weeks = transactions.groupby('customer_id')['week'].unique()

CPU times: user 21.1 s, sys: 1.01 s, total: 22.2 s
Wall time: 21.9 s


In [9]:
transactions.groupby('week')['t_dat'].agg(['min', 'max'])

,min,max
week,,
95,2020-07-15,2020-07-21
96,2020-07-22,2020-07-28
97,2020-07-29,2020-08-04
98,2020-08-05,2020-08-11
99,2020-08-12,2020-08-18
100,2020-08-19,2020-08-25
101,2020-08-26,2020-09-01
102,2020-09-02,2020-09-08
103,2020-09-09,2020-09-15


In [10]:
c2weeks

customer_id
28847241659200          [95, 96, 101, 102]
41318098387474                        [98]
116809474287335                 [101, 103]
200292573348128          [95, 96, 99, 102]
248294615847351                       [96]
                               ...        
18446624797007271432                  [95]
18446630855572834764                 [103]
18446662237889060501                 [100]
18446705133201055310                 [102]
18446737527580148316                 [104]
Name: week, Length: 437365, dtype: object

In [11]:
%%time

c2weeks2shifted_weeks = {}

for c_id, weeks in c2weeks.items():
    c2weeks2shifted_weeks[c_id] = {}
    for i in range(weeks.shape[0]-1):
        c2weeks2shifted_weeks[c_id][weeks[i]] = weeks[i+1]
    c2weeks2shifted_weeks[c_id][weeks[-1]] = test_week

CPU times: user 890 ms, sys: 150 ms, total: 1.04 s
Wall time: 1.04 s


In [12]:
c2weeks2shifted_weeks[28847241659200]

{95: 96, 96: 101, 101: 102, 102: 105}

In [13]:
candidates_last_purchase = transactions.copy()

In [14]:
%%time

weeks = []
for i, (c_id, week) in enumerate(zip(transactions['customer_id'], transactions['week'])):
    weeks.append(c2weeks2shifted_weeks[c_id][week])
    
candidates_last_purchase.week=weeks

CPU times: user 7.81 s, sys: 241 ms, total: 8.05 s
Wall time: 8.07 s


In [15]:
candidates_last_purchase[candidates_last_purchase['customer_id']==272412481300040]

,t_dat,customer_id,article_id,price,sales_channel_id,week
29030503,2020-07-15,272412481300040,778064028,0.008458,1,96
29030504,2020-07-15,272412481300040,816592008,0.016932,1,96
29030505,2020-07-15,272412481300040,621381021,0.033881,1,96
29030506,2020-07-15,272412481300040,817477003,0.025407,1,96
29030507,2020-07-15,272412481300040,899088002,0.025407,1,96
29319533,2020-07-22,272412481300040,885077001,0.008458,1,103
29410772,2020-07-24,272412481300040,850176003,0.029034,2,103
29410773,2020-07-24,272412481300040,875803001,0.064559,2,103
29410774,2020-07-24,272412481300040,892970003,0.020966,2,103
29410775,2020-07-24,272412481300040,854619003,0.020966,2,103


In [16]:
transactions[transactions['customer_id']==272412481300040]

,t_dat,customer_id,article_id,price,sales_channel_id,week
29030503,2020-07-15,272412481300040,778064028,0.008458,1,95
29030504,2020-07-15,272412481300040,816592008,0.016932,1,95
29030505,2020-07-15,272412481300040,621381021,0.033881,1,95
29030506,2020-07-15,272412481300040,817477003,0.025407,1,95
29030507,2020-07-15,272412481300040,899088002,0.025407,1,95
29319533,2020-07-22,272412481300040,885077001,0.008458,1,96
29410772,2020-07-24,272412481300040,850176003,0.029034,2,96
29410773,2020-07-24,272412481300040,875803001,0.064559,2,96
29410774,2020-07-24,272412481300040,892970003,0.020966,2,96
29410775,2020-07-24,272412481300040,854619003,0.020966,2,96


### Bestsellers candidates

In [17]:
mean_price = transactions \
    .groupby(['week', 'article_id'])['price'].mean()

In [18]:
mean_price

week  article_id
95    108775015     0.004729
      108775044     0.008458
      110065001     0.006085
      110065002     0.006085
      111565001     0.004288
                      ...   
104   952267001     0.013732
      952938001     0.048651
      953450001     0.016932
      953763001     0.021885
      956217002     0.059068
Name: price, Length: 196880, dtype: float32

In [19]:
sales = transactions \
    .groupby('week')['article_id'].value_counts() \
    .groupby('week').rank(method='dense', ascending=False) \
    .groupby('week').head(12).rename('bestseller_rank').astype('int8')

In [20]:
sales

week  article_id
95    760084003      1
      866731001      2
      600886001      3
      706016001      4
      372860002      5
                    ..
104   915529003      8
      915529005      9
      448509014     10
      762846027     11
      714790020     12
Name: bestseller_rank, Length: 120, dtype: int8

In [21]:
sales.loc[95]

article_id
760084003     1
866731001     2
600886001     3
706016001     4
372860002     5
610776002     6
877278002     7
547780003     8
817354001     9
827968001    10
866731003    11
866383006    12
Name: bestseller_rank, dtype: int8

In [22]:
bestsellers_previous_week = pd.merge(sales, mean_price, on=['week', 'article_id']).reset_index()
bestsellers_previous_week.week += 1

In [23]:
bestsellers_previous_week.pipe(lambda df: df[df['week']==96])

,week,article_id,bestseller_rank,price
0,96,760084003,1,0.025094
1,96,866731001,2,0.024919
2,96,600886001,3,0.022980
3,96,706016001,4,0.033197
4,96,372860002,5,0.013193
5,96,610776002,6,0.008318
6,96,877278002,7,0.025036
7,96,547780003,8,0.024814
8,96,817354001,9,0.021913
9,96,827968001,10,0.016436


In [24]:
unique_transactions = transactions \
    .groupby(['week', 'customer_id']) \
    .head(1) \
    .drop(columns=['article_id', 'price']) \
    .copy()

In [25]:
unique_transactions

,t_dat,customer_id,sales_channel_id,week
29030503,2020-07-15,272412481300040,1,95
29064059,2020-07-15,1456826891333599,1,95
29067103,2020-07-15,2133687643102426,2,95
29027487,2020-07-15,6010692573790711,1,95
29046403,2020-07-15,6171059100114610,2,95
...,...,...,...,...
31760188,2020-09-22,18435221511488011015,1,104
31782234,2020-09-22,18436859303155335645,1,104
31787251,2020-09-22,18437941771381362708,2,104
31776022,2020-09-22,18438270306572912089,1,104


In [26]:
transactions.drop_duplicates(['week', 'customer_id'])

,t_dat,customer_id,article_id,price,sales_channel_id,week
29030503,2020-07-15,272412481300040,778064028,0.008458,1,95
29064059,2020-07-15,1456826891333599,888294001,0.013542,1,95
29067103,2020-07-15,2133687643102426,843642001,0.042356,2,95
29027487,2020-07-15,6010692573790711,857812010,0.039661,1,95
29046403,2020-07-15,6171059100114610,815447007,0.006763,2,95
...,...,...,...,...,...,...
31760188,2020-09-22,18435221511488011015,573085055,0.033881,1,104
31782234,2020-09-22,18436859303155335645,801447001,0.030492,1,104
31787251,2020-09-22,18437941771381362708,907188001,0.050831,2,104
31776022,2020-09-22,18438270306572912089,751471043,0.033881,1,104


In [27]:
candidates_bestsellers = pd.merge(
    unique_transactions,
    bestsellers_previous_week,
    on='week',
)

In [28]:
test_set_transactions = unique_transactions.drop_duplicates('customer_id').reset_index(drop=True)
test_set_transactions.week = test_week

In [29]:
test_set_transactions

,t_dat,customer_id,sales_channel_id,week
0,2020-07-15,272412481300040,1,105
1,2020-07-15,1456826891333599,1,105
2,2020-07-15,2133687643102426,2,105
3,2020-07-15,6010692573790711,1,105
4,2020-07-15,6171059100114610,2,105
...,...,...,...,...
437360,2020-09-22,18410229429441241008,2,105
437361,2020-09-22,18417769707947924979,2,105
437362,2020-09-22,18418054986721795659,2,105
437363,2020-09-22,18421175435799911749,2,105


In [30]:
candidates_bestsellers_test_week = pd.merge(
    test_set_transactions,
    bestsellers_previous_week,
    on='week'
)

In [31]:
candidates_bestsellers = pd.concat([candidates_bestsellers, candidates_bestsellers_test_week])
candidates_bestsellers.drop(columns='bestseller_rank', inplace=True)

In [32]:
candidates_bestsellers

,t_dat,customer_id,sales_channel_id,week,article_id,price
0,2020-07-22,200292573348128,2,96,760084003,0.025094
1,2020-07-22,200292573348128,2,96,866731001,0.024919
2,2020-07-22,200292573348128,2,96,600886001,0.022980
3,2020-07-22,200292573348128,2,96,706016001,0.033197
4,2020-07-22,200292573348128,2,96,372860002,0.013193
...,...,...,...,...,...,...
5248375,2020-09-22,18438270306572912089,1,105,915529003,0.033439
5248376,2020-09-22,18438270306572912089,1,105,915529005,0.033417
5248377,2020-09-22,18438270306572912089,1,105,448509014,0.041630
5248378,2020-09-22,18438270306572912089,1,105,762846027,0.025005


# Combining transactions and candidates / negative examples

In [33]:
transactions['purchased'] = 1

In [34]:
data = pd.concat([transactions, candidates_last_purchase, candidates_bestsellers])
data.purchased.fillna(0, inplace=True)

In [35]:
data

,t_dat,customer_id,article_id,price,sales_channel_id,week,purchased
29030503,2020-07-15,272412481300040,778064028,0.008458,1,95,1.0
29030504,2020-07-15,272412481300040,816592008,0.016932,1,95,1.0
29030505,2020-07-15,272412481300040,621381021,0.033881,1,95,1.0
29030506,2020-07-15,272412481300040,817477003,0.025407,1,95,1.0
29030507,2020-07-15,272412481300040,899088002,0.025407,1,95,1.0
...,...,...,...,...,...,...,...
5248375,2020-09-22,18438270306572912089,915529003,0.033439,1,105,0.0
5248376,2020-09-22,18438270306572912089,915529005,0.033417,1,105,0.0
5248377,2020-09-22,18438270306572912089,448509014,0.041630,1,105,0.0
5248378,2020-09-22,18438270306572912089,762846027,0.025005,1,105,0.0


In [36]:
data.drop_duplicates(['customer_id', 'article_id', 'week'], inplace=True)

In [37]:
data.purchased.mean()

0.13607582749165664

### Add bestseller information

In [38]:
data = pd.merge(
    data,
    bestsellers_previous_week[['week', 'article_id', 'bestseller_rank']],
    on=['week', 'article_id'],
    how='left'
)

In [39]:
data = data[data.week != data.week.min()]
data.bestseller_rank.fillna(999, inplace=True)

In [40]:
data = pd.merge(data, articles, on='article_id', how='left')
data = pd.merge(data, customers, on='customer_id', how='left')

### Features

Feature 1: week_nr (which week out of the 52 weeks in a year is it)

In [41]:
# data.sample(20)

In [42]:
data['week_nr'] = data['t_dat'].dt.isocalendar().week.to_frame().astype("int32")
data.dtypes

t_dat                           datetime64[ns]
customer_id                             uint64
article_id                               int64
price                                  float32
sales_channel_id                          int8
week                                     int64
purchased                              float64
bestseller_rank                        float64
product_code                             int32
prod_name                                int32
product_type_no                          int32
product_type_name                        int16
product_group_name                        int8
graphical_appearance_no                  int32
graphical_appearance_name                 int8
colour_group_code                        int32
colour_group_name                         int8
perceived_colour_value_id                int32
perceived_colour_value_name               int8
perceived_colour_master_id               int32
perceived_colour_master_name              int8
department_no

In [43]:
data.sort_values(by=['t_dat'])
group = data.groupby('customer_id')
group.head(30)

,t_dat,customer_id,article_id,price,sales_channel_id,week,purchased,bestseller_rank,product_code,prod_name,...,garment_group_no,garment_group_name,detail_desc,FN,Active,club_member_status,fashion_news_frequency,age,postal_code,week_nr
0,2020-07-22,200292573348128,880777001,0.025407,2,96,1.0,999.0,880777,10832,...,1018,12,9213,1,1,0,1,25,63947,30
1,2020-07-22,200292573348128,784332002,0.025407,2,96,1.0,999.0,784332,25312,...,1005,0,7303,1,1,0,1,25,63947,30
2,2020-07-22,200292573348128,827968001,0.016932,2,96,1.0,10.0,827968,1066,...,1002,2,1227,1,1,0,1,25,63947,30
3,2020-07-22,200292573348128,599580086,0.011847,2,96,1.0,999.0,599580,36,...,1018,12,52,1,1,0,1,25,63947,30
4,2020-07-22,248294615847351,720504008,0.031458,1,96,1.0,999.0,720504,1373,...,1016,11,95,-1,-1,0,0,46,8666,30
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17991757,2020-09-22,18438270306572912089,915529003,0.033439,1,105,0.0,8.0,915529,7046,...,1003,3,10909,1,1,0,1,71,116920,39
17991758,2020-09-22,18438270306572912089,915529005,0.033417,1,105,0.0,9.0,915529,7046,...,1003,3,10909,1,1,0,1,71,116920,39
17991759,2020-09-22,18438270306572912089,448509014,0.041630,1,105,0.0,10.0,448509,259,...,1009,5,255,1,1,0,1,71,116920,39
17991760,2020-09-22,18438270306572912089,762846027,0.025005,1,105,0.0,11.0,762846,472,...,1010,6,492,1,1,0,1,71,116920,39


In [44]:
group = data.sort_values(['t_dat'], ascending=True).groupby('customer_id')
data['days_since_last_purchase'] = group['t_dat'].diff()

In [45]:
data['days_since_last_purchase'] = data['days_since_last_purchase'].dt.days

In [46]:
# print(data['time_since_last_purchase'].value_counts().to_string())

In [47]:
# data['time_since_last_purchase'].plot.bar()
# plt.show()

In [48]:
data.dtypes
data.head(20)

,t_dat,customer_id,article_id,price,sales_channel_id,week,purchased,bestseller_rank,product_code,prod_name,...,garment_group_name,detail_desc,FN,Active,club_member_status,fashion_news_frequency,age,postal_code,week_nr,days_since_last_purchase
0,2020-07-22,200292573348128,880777001,0.025407,2,96,1.0,999.0,880777,10832,...,12,9213,1,1,0,1,25,63947,30,0.0
1,2020-07-22,200292573348128,784332002,0.025407,2,96,1.0,999.0,784332,25312,...,0,7303,1,1,0,1,25,63947,30,0.0
2,2020-07-22,200292573348128,827968001,0.016932,2,96,1.0,10.0,827968,1066,...,2,1227,1,1,0,1,25,63947,30,0.0
3,2020-07-22,200292573348128,599580086,0.011847,2,96,1.0,999.0,599580,36,...,12,52,1,1,0,1,25,63947,30,0.0
4,2020-07-22,248294615847351,720504008,0.031458,1,96,1.0,999.0,720504,1373,...,11,95,-1,-1,0,0,46,8666,30,0.0
5,2020-07-22,248294615847351,471714002,0.007847,1,96,1.0,999.0,471714,243,...,16,635,-1,-1,0,0,46,8666,30,0.0
6,2020-07-22,248294615847351,878987003,0.025169,1,96,1.0,999.0,878987,4158,...,16,2139,-1,-1,0,0,46,8666,30,0.0
7,2020-07-22,248294615847351,337991001,0.023593,1,96,1.0,999.0,337991,3818,...,1,778,-1,-1,0,0,46,8666,30,0.0
8,2020-07-22,272412481300040,885077001,0.008458,1,96,1.0,999.0,885077,9486,...,1,10437,1,1,0,1,48,333369,30,0.0
9,2020-07-22,649760207043851,878502001,0.022017,1,96,1.0,999.0,878502,2008,...,0,11805,1,1,0,1,35,38525,30,0.0


In [49]:
data.sort_values(['week', 'customer_id'], inplace=True)
data.reset_index(drop=True, inplace=True)

In [50]:
train = data[data.week != test_week]
test = data[data.week==test_week].drop_duplicates(['customer_id', 'article_id', 'sales_channel_id']).copy()

In [51]:
train_baskets = train.groupby(['week', 'customer_id'])['article_id'].count().values

In [52]:
columns_to_use = ['article_id', 'product_type_no', 'graphical_appearance_no', 'colour_group_code', 'perceived_colour_value_id',
'perceived_colour_master_id', 'department_no', 'index_code',
'index_group_no', 'section_no', 'garment_group_no', 'FN', 'Active',
'club_member_status', 'fashion_news_frequency', 'age', 'postal_code', 'bestseller_rank', 'week_nr', 'days_since_last_purchase']

In [53]:
%%time

train_X = train[columns_to_use]
train_y = train['purchased']

test_X = test[columns_to_use]

CPU times: user 368 ms, sys: 290 ms, total: 658 ms
Wall time: 656 ms


# Model training

In [54]:
from lightgbm.sklearn import LGBMRanker

In [55]:
ranker = LGBMRanker(
    objective="lambdarank",
    metric="ndcg",
    boosting_type="dart",
    n_estimators=1,
    importance_type='gain',
    verbose=10
)

In [56]:
%%time

ranker = ranker.fit(
    train_X,
    train_y,
    group=train_baskets,
)

[LightGBM] [Debug] Dataset::GetMultiBinFromSparseFeatures: sparse rate 0.883563
[LightGBM] [Debug] Dataset::GetMultiBinFromAllFeatures: sparse rate 0.185438
[LightGBM] [Debug] init for col-wise cost 0.548644 seconds, init for row-wise cost 1.135975 seconds
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.951844 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Debug] Using Dense Multi-Val Bin
[LightGBM] [Info] Total Bins 1155
[LightGBM] [Info] Number of data points in the train set: 11381612, number of used features: 20
[LightGBM] [Debug] Trained a tree with leaves = 31 and depth = 8
CPU times: user 32.3 s, sys: 2.76 s, total: 35.1 s
Wall time: 12.2 s


In [57]:
for i in ranker.feature_importances_.argsort()[::-1]:
    print(columns_to_use[i], ranker.feature_importances_[i]/ranker.feature_importances_.sum())

bestseller_rank 0.9727993015425599
week_nr 0.021548937507524887
days_since_last_purchase 0.005465874017936655
age 9.310525519613264e-05
garment_group_no 6.62028356045802e-05
product_type_no 2.6578841177841465e-05
index_code 0.0
graphical_appearance_no 0.0
colour_group_code 0.0
perceived_colour_value_id 0.0
perceived_colour_master_id 0.0
department_no 0.0
section_no 0.0
index_group_no 0.0
FN 0.0
Active 0.0
club_member_status 0.0
fashion_news_frequency 0.0
postal_code 0.0
article_id 0.0


# Calculate predictions

In [58]:
%time

test['preds'] = ranker.predict(test_X)

c_id2predicted_article_ids = test \
    .sort_values(['customer_id', 'preds'], ascending=False) \
    .groupby('customer_id')['article_id'].apply(list).to_dict()

bestsellers_last_week = \
    bestsellers_previous_week[bestsellers_previous_week.week == bestsellers_previous_week.week.max()]['article_id'].tolist()

CPU times: user 4 µs, sys: 0 ns, total: 4 µs
Wall time: 10.5 µs


# Create submission

In [59]:
sub = pd.read_csv('/kaggle/input/h-and-m-personalized-fashion-recommendations/sample_submission.csv')

In [60]:
%%time
preds = []
for c_id in customer_hex_id_to_int(sub.customer_id):
    pred = c_id2predicted_article_ids.get(c_id, [])
    pred = pred + bestsellers_last_week
    preds.append(pred[:12])

CPU times: user 6.29 s, sys: 335 ms, total: 6.62 s
Wall time: 6.63 s


In [61]:
preds = [' '.join(['0' + str(p) for p in ps]) for ps in preds]
sub.prediction = preds

In [62]:
sub_name = 'basic_model_submission'
sub.to_csv(f'{sub_name}.csv.gz', index=False)